# Analysing decisions in data analysis

H. Sherry Zhang (University of Texas at Austin)  
Roger D. Peng (University of Texas at Austin)

this is the abstract

# 1. Introduction

Something about “analysis review” - Roger thinks it’s a better to have a new word for this.

provide a baseline understand - place to start

demonstrate - analytically homogeneous - the table won’t look like that

In this work, we design a tabular format to record the choices made by analysts during data analysis. Using large language models, we automatically extract these choices from a set of research papers focused on specific topics, e.g. air pollution modelling. This allows us to analyze these choices as data – tracking how they’ve changed over time or query the possible methodologies used in similar studies. We also introduce a workflow to cluster paper based on decision similarity, using both the decisions themselves and the justifications authors provide for their choices.

# 2. Background

Data analysis as an complicated, iterative process to make sense \[ref\] of the data collected. The iterative process of formulating hypothesis Jun et al. ([2022](#ref-jun2022)).

Choices are made at nearly every stage of data analysis, ranging from variable pre-processing variables, variable and lag selection in model formulation, to the specification of smoothing parameter during model construction. These possible choices contribute to what Gelman and Loken ([2014](#ref-gelman2014)) describe as the “garden of forking paths”. These choices can introduce substantial variability in results, which has been demonstrated in many-analyst experiments, where independent teams analyzing the same dataset to answer a pre-defined research question often arrive at markedly different conclusions. A prominent example is Silberzahn et al. ([2018](#ref-silberzahn2018)) where researchers reported a wide range of point estimates and 95% confidence intervals for the effect of soccer players’ skin tone on the number of red cards awarded by referees (odds ratio from 0.89 to 2.93). Similar findings have emerged in other domains, including structural equation modeling ([Sarstedt et al. 2024](#ref-sarstedt2024)), applied microeconomics ([Huntington-Klein et al. 2021](#ref-huntington-klein2021)), neuroimaging ([Botvinik-Nezer et al. 2020](#ref-botvinik-nezer2020)), and ecology and evolutionary biology ([Gould et al. 2025](#ref-gould2025)).

Another line of work focuses on developing software tools to support analysts in making more informed decisions. For example, the `Tisane` package ([Jun et al. 2022](#ref-jun2022)) integrates conceptual ideas, such as DAGs, and modelling structure (group/ cluster/ hierarchical structure), to assist junior researchers in specifying GLM and GLMM model. The `DeclareDesign` package ([Blair et al. 2019](#ref-blair2019)) introduces the MIDA framework for researchers to declare, diagnose, and redesign their analyses to produce a distribution of the statistic of interest. This approach has been applied in randomized controlled trial ([Bishop and Hulme 2024](#ref-bishop2024)) .

The `multiverse` package

-   facilitates the specification and execution of multiple parallel choices for sensitivity analysis, allowing researchers to systematically explore how different choices affect results and to report the range of plausible outcomes that arise from alternative analytic paths.

Study decisions in data analysis:

-   interview analysts and researchers to provide recommendation for data analysis practices ([Kale, Kay, and Hullman 2019](#ref-kale2019); [Alspaugh et al. 2019](#ref-alspaugh2019); [Yang Liu, Althoff, and Heer 2020](#ref-liu2020)).
-   Yang Liu, Althoff, and Heer ([2020](#ref-liu2020)) provides visualization to communicate the decision processes through the Analytic Decision Graphs (ADG)
-   Simson et al. ([2025](#ref-simson2025)) conducts a participatory AI study to demonstrate the “garden of forking paths” of decisions in data analysis and how it affects ML fairness

# 3. Extracting decisions from data analysis

## 3.1 Decisions in data analysis

-   what constitute a decision in data analysis
-   adapt from the tidy data principle - each row is a decision Wickham ([2014](#ref-wickham2014))
-   some decisions are related to how the variable is estimated spatially and temporally
-   model level decisions on how the model is estimated spatially (for multi-site analyses) and/or temporally (different treatments for years or seasons)
-   extract the exact text from the paper

An example decisions may look as follows:

| Paper | ID | Model | variable | method | parameter | type | reason | decision |
|--------|--------|--------|--------|--------|--------|--------|--------|--------|
| ostro | 1 | Poisson regression | temperature | smoothing spline | degree of freedom | parameter | NA | 3 degree of freedom |
| ostro | 2 | Poisson regression | temperature | smoothing spline | degree of freedom | temporal | NA | 1-day lag |
| ostro | 3 | Poisson regression | relative humidity | LOESS | smoothing parameter | parameter | to minimize Akaike’s Information Criterion | NA |
| ostro | 4 | Poisson regression | model | NA | NA | spatial | to account for variation among cities | separate regression models fit in each city |

However, decisions statements are often implicit, and the justifications may not directly align with the decisions themselves. We identify four common anomalies:

1.  **Authors may combine multiple decisions into a single sentence** for coherence and conciseness of the writing. Consider the following excerpt from Ostro et al. ([2006](#ref-ostro2006)):

> Other covariates, such as day of the week and smoothing splines of 1-day lags of average temperature and humidity (each with 3 df), were also included in the model because they may be associated with daily mortality and are likely to vary over time in concert with air pollution levels.

This sentence contains four decisions: two for temperature (the temporal lag and the smoothing spline parameter) and two for humidity. These decisions should be structured as separate entries.

1.  **The justification does not directly address the decision choice.** In the example above, the stated rationale (“and are likely to vary over time in concert with air pollution levels”) supports the general inclusion of temporal lags but does not justify the specific choice of 1-day lag over alternatives, such as 2-day average of lags 0 and 1 (lag01) and single-day lag of 2 days (lag2). As such, the reason field should be recorded as NA.

2.  **Some decisions may be omitted because they are data-driven**. For instance, Katsouyanni et al. ([2001](#ref-katsouyanni2001)) states:

> The inclusion of lagged weather variables and the choice of smoothing parameters for all of the weather variables were done by minimizing Akaike’s information criterion.

In this case, while the method of selection (minimizing AIC) is specified, the actual degree of freedom used is not. Such data-driven decisions may be recorded with “NA” in the decision field, but the reason field should still be recorded as “by minimizing Akaike’s information criterion”

1.  **Information required to interpret the decision may be distributed across multiple sections**. In the previous example, “weather variables” refers to mean temperature and relative humidity, as defined earlier in the text. This requires cross-referencing across sections to identify the correct variables associated with each modeling choice.

## 3.2 Automatic reading of literature with LLM

While decisions can be extracted manually from the literature, this process is labor-intensive and time-consuming. Recent advances in Large Language Models (LLMs) have demonstrated potential for automating the extraction of structured information from unstructured text \[ref\]. In this work, we use LLMs to automatically identify decisions made by authors during their data analysis processes.

Text recognition from PDF document relies on Optical Character Recognition (OCR) to convert scanned images into machine-readable text – capability currently offered by Antropic Claude and Google Gemini. We instruct the LLM to generate a markdown file containing a JSON block that records extracted decisions, which can then be read into statistical software for further analysis. The exact prompt feed to the LLM is provided in the Appendix. The `ellmer` package ([Wickham, Cheng, and Jacobs 2025](#ref-ellmer)) in R is used to connect to the Gemini and Claude API, providing the PDF attachment and the prompt in a markdown file as inputs. **?@fig-llm** shows the overall workflow for decision extract using LLMs.

## 3.3 Review the LLM output

-   something about result validation of LLM output
-   The sensitivity of the two models to the prompt and the model parameters, such as temperature and seed, is discussed in <a href="#sec-sensitivity" class="quarto-xref">Section 5.2</a>.

The shiny app is designed to provide users a visual interface to review and edit the decisions extracted by the LLM from the literature. The app allows three actions from the users: 1) *overwrite* – modify the content of a particular cell, equivalently `dplyr::mutate(xxx = ifelse(CONDITION, "yyy" , xxx))`, 2) *delete* – remove a particular cell, `dplyr::filter(!(CONDITION))`, and 3) *add* – manually enter a decision, `dplyr::bind_rows()`. <a href="#fig-shiny" class="quarto-xref">Figure 1</a> illustrates the *overwrite* action in the Shiny application, where users interactively filter the data and preview the rows affected by their edits—in this case, changing the model entry from “generalized additive Poisson time series regression” to the less verbose “Poisson regression”. Upon confirmation, the corresponding `tidyverse` code is generated, and users can download the edited table and incorporate the code into their R script.

In [ ]:
knitr::include_graphics("figures/shiny.png")

# 4. Calculating paper similarity

-   pre-processing
    -   standardize statistical methods its corresponding parameters (LOESS, smoothing spline, etc)
    -   group variables into broader categories: time, temperature, humidity, PM
-   identify the most frequent analysis decisions across papers
-   retain only papers that report more than x such decisions
-   measure similarity between decisions and their justificaiton using NLP
    -   word embedding with attention mechanism, instead of bag of word,
    -   specific NLP models (default to `bert-base-uncased`), aggregation methods from word to text
-   compute paper similarity score for each paper pair by aggregating decision-level compoarisons
    -   check/ report on the number of decisions compared in each paper pair
-   similarity score can serve as the distance matrix to cluster papers by their similarity on decision choices

# 5. Results

## 5.1 Air pollution mortality modelling

Decision quality summary

-   look at for one type of decision (time) - what are the choices made by different papers
-   look at whether decisions changes across time
-   Visualize the decision database: apply clustering algorithm and visualize the database through `sigma.js`

## 5.2 Sensitivity analysis

sensitivity of the pipeline: 1) LLM, 2) text model, 3) prompt, 4) LLM parameters

-   standard `BERT` ([Devlin et al. 2019](#ref-devlin2019)), `Roberta` ([Yinhan Liu et al., n.d.](#ref-liu)): trained on a much larger dataset (160GB v.s. BERT’s 15GB), `transformer-xl` ([Dai et al., n.d.](#ref-dai)), `xlnet` by Google Brain ([Yang et al., n.d.](#ref-yang)), and two domain-trained BERT models: `sciBert` ([Beltagy, Lo, and Cohan 2019](#ref-beltagy2019)) and `bioBert`([Lee et al. 2020](#ref-lee2020)), trained on PubMed and PMC data.

-   A section on reproducibility of LLM outputs: prompt experiment (see if there are papers discussing this: https://arxiv.org/pdf/2406.06608)

# 6. Discussion

-   Only prompting engineering is used to extract decisions from the literature. We expect that fine-tuning the model on statistical or domain-specific literature to yield more robust performance on the same document, though it would require substantially more training effort.
-   people from the NYU-LMU workshop are interested to have code script attached as well because people can do one thing in the script but report another in the paper - it would be interesting to compare the paper and the script with some syntax extraction.
-   Validation of the output:

the nature of the task: Our task involve a reasoning component in that it requires casual reasoning to identify the decisions made by the authors, and its justification/ rationale, rather than purely summarizing the text through pattern-matching.

# 7. Reference

Alspaugh, Sara, Nava Zokaei, Andrea Liu, Cindy Jin, and Marti A. Hearst. 2019. “Futzing and Moseying: Interviews with Professional Data Analysts on Exploration Practices.” *IEEE Transactions on Visualization and Computer Graphics* 25 (1): 22–31. <https://doi.org/10.1109/TVCG.2018.2865040>.

Beltagy, Iz, Kyle Lo, and Arman Cohan. 2019. “Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing and the 9th International Joint Conference on Natural Language Processing (EMNLP-IJCNLP).” In, 3613–18. Hong Kong, China: Association for Computational Linguistics. <https://doi.org/10.18653/v1/D19-1371>.

Bishop, Dorothy V. M., and Charles Hulme. 2024. “When Alternative Analyses of the Same Data Come to Different Conclusions: A Tutorial Using DeclareDesign With a Worked Real-World Example.” *Advances in Methods and Practices in Psychological Science* 7 (3): 25152459241267904. <https://doi.org/10.1177/25152459241267904>.

Blair, Graeme, Jasper Cooper, Alexander Coppock, and Macartan Humphreys. 2019. “Declaring and Diagnosing Research Designs.” *American Political Science Review* 113 (3): 838–59. <https://doi.org/10.1017/S0003055419000194>.

Botvinik-Nezer, Rotem, Felix Holzmeister, Colin F. Camerer, Anna Dreber, Juergen Huber, Magnus Johannesson, Michael Kirchler, et al. 2020. “Variability in the Analysis of a Single Neuroimaging Dataset by Many Teams.” *Nature* 582 (7810): 84–88. <https://doi.org/10.1038/s41586-020-2314-9>.

Dai, Zihang, Zhilin Yang, Yiming Yang, Jaime Carbonell, Quoc V. Le, and Ruslan Salakhutdinov. n.d. “Transformer-XL: Attentive Language Models Beyond a Fixed-Length Context.” <https://doi.org/10.48550/arXiv.1901.02860>.

Devlin, Jacob, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova. 2019. “NAACL-HLT 2019.” In, edited by Jill Burstein, Christy Doran, and Thamar Solorio, 41714186. Minneapolis, Minnesota: Association for Computational Linguistics. <https://doi.org/10.18653/v1/N19-1423>.

Gelman, Andrew, and Eric Loken. 2014. “The Statistical Crisis in Science.” *American Scientist* 102 (6): 460–65. <https://www.proquest.com/docview/1616141998/abstract/5E050DCE82414037PQ/1>.

Gould, Elliot, Hannah S. Fraser, Timothy H. Parker, Shinichi Nakagawa, Simon C. Griffith, Peter A. Vesk, Fiona Fidler, et al. 2025. “Same Data, Different Analysts: Variation in Effect Sizes Due to Analytical Decisions in Ecology and Evolutionary Biology.” *BMC Biology* 23 (1): 35. <https://doi.org/10.1186/s12915-024-02101-x>.

Huntington-Klein, Nick, Andreu Arenas, Emily Beam, Marco Bertoni, Jeffrey R. Bloem, Pralhad Burli, Naibin Chen, et al. 2021. “The Influence of Hidden Researcher Decisions in Applied Microeconomics.” *Economic Inquiry* 59 (3): 944–60. <https://doi.org/10.1111/ecin.12992>.

Jun, Eunice, Melissa Birchfield, Nicole De Moura, Jeffrey Heer, and René Just. 2022. “Hypothesis Formalization: Empirical Findings, Software Limitations, and Design Implications.” *ACM Transactions on Computer-Human Interaction* 29 (1): 1–28. <https://doi.org/10.1145/3476980>.

Kale, Alex, Matthew Kay, and Jessica Hullman. 2019. “Decision-Making Under Uncertainty in Research Synthesis: Designing for the Garden of Forking Paths.” In, 114. CHI ’19. New York, NY, USA: Association for Computing Machinery. <https://doi.org/10.1145/3290605.3300432>.

Katsouyanni, Klea, Giota Touloumi, Evangelia Samoli, Alexandros Gryparis, Alain Le Tertre, Yannis Monopolis, Giuseppe Rossi, et al. 2001. “Confounding and Effect Modification in the Short-Term Effects of Ambient Particles on Total Mortality: Results from 29 European Cities Within the APHEA2 Project.” *Epidemiology* 12 (5): 521. <https://journals.lww.com/epidem/fulltext/2001/09000/confounding_and_effect_modification_in_the.11.aspx>.

Lee, Jinhyuk, Wonjin Yoon, Sungdong Kim, Donghyeon Kim, Sunkyu Kim, Chan Ho So, and Jaewoo Kang. 2020. “BioBERT: A Pre-Trained Biomedical Language Representation Model for Biomedical Text Mining.” Edited by Jonathan Wren. *Bioinformatics* 36 (4): 1234–40. <https://doi.org/10.1093/bioinformatics/btz682>.

Liu, Yang, Tim Althoff, and Jeffrey Heer. 2020. “Paths Explored, Paths Omitted, Paths Obscured: Decision Points & Selective Reporting in End-to-End Data Analysis.” In, 114. CHI ’20. New York, NY, USA: Association for Computing Machinery. <https://doi.org/10.1145/3313831.3376533>.

Liu, Yinhan, Myle Ott, Naman Goyal, Jingfei Du, Mandar Joshi, Danqi Chen, Omer Levy, Mike Lewis, Luke Zettlemoyer, and Veselin Stoyanov. n.d. “RoBERTa: A Robustly Optimized BERT Pretraining Approach.” <https://doi.org/10.48550/arXiv.1907.11692>.

Ostro, Bart, Rachel Broadwin, Shelley Green, Wen-Ying Feng, and Michael Lipsett. 2006. “Fine Particulate Air Pollution and Mortality in Nine California Counties: Results from CALFINE.” *Environmental Health Perspectives* 114 (1): 29–33. <https://doi.org/10.1289/ehp.8335>.

Sarstedt, Marko, Susanne J. Adler, Christian M. Ringle, Gyeongcheol Cho, Adamantios Diamantopoulos, Heungsun Hwang, and Benjamin D. Liengaard. 2024. “Same Model, Same Data, but Different Outcomes: Evaluating the Impact of Method Choices in Structural Equation Modeling.” *Journal of Product Innovation Management* 41 (6): 1100–1117. <https://doi.org/10.1111/jpim.12738>.

Silberzahn, R., E. L. Uhlmann, D. P. Martin, P. Anselmi, F. Aust, E. Awtrey, Š. Bahník, et al. 2018. “Many Analysts, One Data Set: Making Transparent How Variations in Analytic Choices Affect Results.” *Advances in Methods and Practices in Psychological Science* 1 (3): 337–56. <https://doi.org/10.1177/2515245917747646>.

Simson, Jan, Fiona Draxler, Samuel Mehr, and Christoph Kern. 2025. “Preventing Harmful Data Practices by Using Participatory Input to Navigate the Machine Learning Multiverse.” In, 130. CHI ’25. New York, NY, USA: Association for Computing Machinery. <https://doi.org/10.1145/3706598.3713482>.

Wickham, Hadley. 2014. “Tidy Data.” *Journal of Statistical Software* 59 (September): 1–23. <https://doi.org/10.18637/jss.v059.i10>.

Wickham, Hadley, Joe Cheng, and Aaron Jacobs. 2025. *Ellmer: Chat with Large Language Models*. <https://CRAN.R-project.org/package=ellmer>.

Yang, Zhilin, Zihang Dai, Yiming Yang, Jaime Carbonell, Ruslan Salakhutdinov, and Quoc V. Le. n.d. “XLNet: Generalized Autoregressive Pretraining for Language Understanding.” <https://doi.org/10.48550/arXiv.1906.08237>.